In [1]:
import requests
import dataset
import re
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pywren

In [37]:
db = dataset.connect('sqlite:///books.db')
base_url = 'http://books.toscrape.com/'

def scrape_books(url_list): 
    r = requests.get(url)
    html_soup = BeautifulSoup(r.text, 'html.parser')
    books=html_soup.select('article.product_pod')
    book_list_id = []
    #book = books
    for book in books:
        book_url = book.find('h3').find('a').get('href')
        book_url = urljoin(url, book_url)
        path = urlparse(book_url).path
        book_id = path.split('/')[2]
        book_list_id.append(book_id)
    return book_list_id
        # Upsert tries to update first and then insert instead
#         db['books'].upsert({'book_id' : book_id,
#                             'last_seen' : datetime.now()
#                             }, ['book_id'])


In [43]:
#book_id_list = pywren.get_all_results(pwex.map(scrape_books,url_list[1:2]))
db['books'].upsert({'book_id' : book_id_list[0],'last_seen' : datetime.now()}, ['book_id'])

#book_id_list[0]

True

In [ ]:
# Scrape the pages in the catalogue
pwex = pywren.default_executor()
url = base_url
url_list = []
#inp = input('Do you wish to re-scrape the catalogue (y/n)? ')
inp == 'y'
while True and inp == 'y':
    url_list.append(url)
    r = requests.get(url)
    html_soup = BeautifulSoup(r.text, 'html.parser')
    next_a = html_soup.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url = urljoin(url, next_a[0].get('href'))

book_id_list = pywren.get_all_results(pwex.map(scrape_books,url_list[1:2]))

In [47]:
db['books'].upsert({'book_id' : book_id_list[0],'last_seen' : datetime.now()}, ['book_id'])

True

In [24]:
#books = db['books'].find(order_by=['last_seen'])
book_id_list = book_id_list[0]
# for book in books:
#     book_id = book['book_id']
#     book_id_list.append(book_id)

In [26]:
#pwex.map(func = scrape_book1, iterdata =['wall-and-piece_971'])
#pywren.get_all_results(pwex.map(scrape_book1, book_all[1]))

aladdin-and-his-wonderful-lamp_973
worlds-elsewhere-journeys-around-shakespeares-globe_972


In [30]:
def scrape_book1(book_id_list):
        book_list = []
        book_id = book_id_list
        book_url = base_url + 'catalogue/{}'.format(book_id)
        print('Now scraping book:', book_url)
        r = requests.get(book_url)
        r.encoding = 'utf-8'
        html_soup = BeautifulSoup(r.text, 'html.parser')
        main = html_soup.find(class_='product_main')
        book = {}
        book['book_id'] = book_id
        book['title'] = main.find('h1').get_text(strip=True)
        book['price'] = main.find(class_='price_color').get_text(strip=True)
        book['stock'] = main.find(class_='availability').get_text(strip=True)
        book['rating'] = ' '.join(main.find(class_='star-rating') \
                            .get('class')).replace('star-rating', '').strip()
        book['img'] = html_soup.find(class_='thumbnail').find('img').get('src')
        desc = html_soup.find(id='product_description')
        book['description'] = ''
        if desc:
            book['description'] = desc.find_next_sibling('p') \
                                      .get_text(strip=True)
        book_product_table = html_soup.find(text='Product Information').find_next('table')
        for row in book_product_table.find_all('tr'):
            header = row.find('th').get_text(strip=True)
            # Since we'll use the header as a column, clean it a bit
            # to make sure SQLite will accept it
            header = re.sub('[^a-zA-Z]+', '_', header)
            value = row.find('td').get_text(strip=True)
            book[header] = value
        book_list.append(book)
        return book_list
all_books = pywren.get_all_results(pwex.map(scrape_book1,book_id_list))
db['book_info'].upsert(all_books[0], ['book_id'])

[[{'book_id': 'aladdin-and-his-wonderful-lamp_973',
   'title': 'Aladdin and His Wonderful Lamp',
   'price': '£53.13',
   'stock': 'In stock (19 available)',
   'rating': 'Three',
   'img': '../../media/cache/a8/3c/a83c460fab82f35a37c0846729485547.jpg',
   'description': 'Planning on seeing Aladdin the Disney musical? Read the classic story of "Aladdin and his Wonderful Lamp" in this sparkling new edition by Harpendore. Synopsis: Aladdin is a poor boy who lives with his mother in the ancient kingdom of Cathay. One day a strange man appears offering untold riches! Aladdin is not so sure, yet he\'s sorely tempted. Join Aladdin on his incredib Planning on seeing Aladdin the Disney musical? Read the classic story of "Aladdin and his Wonderful Lamp" in this sparkling new edition by Harpendore. Synopsis: Aladdin is a poor boy who lives with his mother in the ancient kingdom of Cathay. One day a strange man appears offering untold riches! Aladdin is not so sure, yet he\'s sorely tempted. Joi